# Graph Generic Cyher or QA Graph Cyher are suitable fetching the exact nodes relating to graph

In [1]:
import pinecone
import openai
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from langchain.embeddings.base import Embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()
neo4j_url = os.getenv('NEO4J_URL')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')
openai_key = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = openai_key
pinecone_api_key = os.getenv('PINECONE_KEY')

embeddings = OpenAIEmbeddings()

/Users/rc/miniforge3/envs/chat/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Step 1. Get names of nodes, edges, labels

In [2]:
# Get all the nodes names

driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))
# Divide the graph into trunks
def get_pair_nodes (tx):
    pairs_of_nodes = []
    for record in tx.run("MATCH (a)-[r]->(b) RETURN labels(a), a.name, type(r), labels(b), b.name"):
        pair_node = {
            "start_node_name": record["a.name"],
            "start_node_label":record["labels(a)"][0],
            "end_node_name": record["b.name"],
            "end_node_label":record["labels(b)"][0],
            "edge": record["type(r)"]
        }
        pairs_of_nodes.append(pair_node)
        # print(record["labels(a)"][0], record["a.name"], record["type(r)"], record["labels(b)"][0], record["b.name"])
    return pairs_of_nodes


In [3]:
with driver.session() as session:
    pairs_of_nodes = session.execute_read(get_pair_nodes)
driver.close()
pairs_of_nodes

[{'start_node_name': 'Review_project_portfolio_balance_for_2020_Q1',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'Portfolio_management',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Conduct_project_assurance_readiness_for_Brexit_Programme',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Identify_best_practice_regulatory_project_as_examplar',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Identify_best_practice_regulatory_project_as_examplar',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Support',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Cross-project_review_of_plastic_dependencies',
  'start_node_label': 'Areas_of_support',


# Step 2: Build vectors

In [4]:
node_names = set()
node_types = set()
edge_types = set()
for p_n in pairs_of_nodes:
    node_names.add(p_n["start_node_name"])
    node_names.add(p_n["end_node_name"])
    
    node_types.add(p_n["start_node_label"])
    node_types.add(p_n["end_node_label"])
    
    edge_types.add(p_n["edge"])

In [5]:
node_vectors = []
for n in node_names:
    ele_info = {
        "value": embeddings.embed_documents([n]),
        "meta_data": {
            "source":"ps-graph-01",
            "info_type": "node_names",
            "name":n
        },
    } 
    node_vectors.append(ele_info)

for n in node_types:
    ele_info = {
        "value": embeddings.embed_documents([n]),
        "meta_data": {
            "source":"ps-graph-01",
            "info_type": "node_types",
            "name":n
        },
    } 
    node_vectors.append(ele_info)

for n in edge_types:
    ele_info = {
        "value": embeddings.embed_documents([n]),
        "meta_data": {
            "source":"ps-graph-01",
            "info_type": "edges",
            "name":n
        },
    } 
    node_vectors.append(ele_info)
node_vectors[:5]

[{'value': [[-0.019000083305011083,
    -0.012543946568584553,
    -0.007241436317251599,
    -0.03947347050180588,
    -0.026185922458242236,
    0.030689234070575656,
    -0.029382717014240375,
    -0.0024966276378142528,
    -0.024837708227383556,
    0.00708159621692266,
    0.003464353055671122,
    0.003770133499662599,
    0.0065117329691243946,
    -0.002296827978064369,
    0.0010363523668392625,
    0.0017981972869949194,
    0.026227619632765636,
    -0.029994279299207196,
    -0.011883738511329678,
    -0.012988718292812623,
    -0.027144961197570712,
    0.004416442033081717,
    0.003579020751271756,
    -0.012940070657879414,
    0.0008296031197944848,
    -0.004242703340239602,
    0.0004760445178091281,
    -0.030383452928092247,
    -0.0013334457830563755,
    0.00740822548100648,
    0.027881613143462568,
    -0.01617856331972353,
    0.0085479524422643,
    -0.004583231196836599,
    -0.00891627841521023,
    -0.03071703218692459,
    -0.021071050423587156,
    0.01

In [6]:
# Setting of pinecone
env_name = "us-west4-gcp-free"
index_name = "test-chatbot-ran"
pinecone.init(api_key=pinecone_api_key,environment=env_name)
index = pinecone.Index(index_name)

# upload vectors 
my_namespace = 'graph'
index.delete(deleteAll='true', namespace=my_namespace)

pinecone_vectors = []
for idx, n_v in enumerate(node_vectors):
    pinecone_vectors.append((str(idx), n_v["value"], n_v["meta_data"]))
upsert_response = index.upsert(vectors=pinecone_vectors, namespace=my_namespace)

# Step 3: Add additional hint on top of original question


In [7]:
prompt = "What types of business value does Unilever create?"

In [9]:
env_name = "us-west4-gcp-free"
index_name = "test-chatbot-ran"
pinecone.init(api_key=pinecone_api_key,environment=env_name)
index = pinecone.Index(index_name)

vectorstore = Pinecone(index , embeddings.embed_query, "name", namespace=my_namespace)

In [11]:
docs = vectorstore.similarity_search(prompt, k=10)
docs

[Document(page_content='Value_Unilever_create', metadata={'info_type': 'node_types', 'source': 'ps-graph-01'}),
 Document(page_content='Business_Transformation_themes', metadata={'info_type': 'node_types', 'source': 'ps-graph-01'}),
 Document(page_content='Business_disruption_', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='1._Winning_with_brands_and_innovation', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='Business_transformation_portfolio', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='7._Consumer_insight_', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='Supply_chain_strategy', metadata={'info_type': 'node_types', 'source': 'ps-graph-01'}),
 Document(page_content='4._Marketing', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='Divestment_', metadata={'info_type': 'node_names', 'so

In [15]:
related_node_names = ""
related_node_types = ""
related_edges = ""
for d in docs: 
    if d.metadata["info_type"] == "node_names":
        related_node_names+=d.page_content + ", "
    
    if d.metadata["info_type"] == "node_types":
        related_node_types+=d.page_content + ", "
    
    if d.metadata["info_type"] == "edges":
        related_edges+=d.page_content + ", "

print(related_node_names)
print(related_node_types) 
print(related_edges)

Business_disruption_, 1._Winning_with_brands_and_innovation, Business_transformation_portfolio, 7._Consumer_insight_, 4._Marketing, Divestment_, 1._Consumer_benefits, 
Value_Unilever_create, Business_Transformation_themes, Supply_chain_strategy, 



In [18]:
additional_hint = f"""Hint: Please refer the names of nodes:{related_node_names} or the labels of nodes: {related_node_types} or the edges type: {related_edges} if necessary"""
additional_hint

'Hint: Please refer the names of nodes:Business_disruption_, 1._Winning_with_brands_and_innovation, Business_transformation_portfolio, 7._Consumer_insight_, 4._Marketing, Divestment_, 1._Consumer_benefits,  or the labels of nodes: Value_Unilever_create, Business_Transformation_themes, Supply_chain_strategy,  or the edges type:  if necessary'

# Add additional hint on top of QA graph

In [21]:
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.chat_models import ChatOpenAI

graph = Neo4jGraph(
    url=neo4j_url, 
    username=neo4j_user, 
    password=neo4j_password
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

result = chain.run(prompt + additional_hint)
result



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Value_Unilever_create)-[:enabled_by]->(b:Business_Transformation_themes)-[:enables]->(s:Supply_chain_strategy)
RETURN DISTINCT s.labels
Full Context:
[]

> Finished chain.


"Unilever creates various types of business value through its focus on business disruption, winning with brands and innovation, consumer insight, marketing, divestment, and consumer benefits. These efforts are part of the company's overall business transformation themes and supply chain strategy, which aim to create value for both the company and its customers."